In [ ]:
import numpy as np
import sys
import os
from pathlib import Path
import suite2p
from suite2p import default_settings
from suite2p.run_s2p import logger_setup

db = {"data_path": ["/media/carsen/disk2/test_suite2p/GT1/"], 
      "nplanes": 1, # each tiff has these many planes in sequence
      "nchannels": 2, # each tiff has these many channels per plane
      }
db["save_path0"] = db["data_path"][0]

# initialize logger
logger_setup(db["save_path0"])

# initialize settings
settings = default_settings()
settings["registration"]["align_by_chan2"] = True # optional - use anatomical chan for reg
settings["run"]["do_registration"] = 1
settings["fs"] = 10
settings["tau"] = 1.0 # timescale of gcamp to use for deconvolution
settings["detection"]["threshold_scaling"] = 1.0
settings["detection"]["algorithm"] = "sparsery"
settings["torch_device"] = "cuda" # use mps for mac, cuda for nvidia gpu, or "cpu"


run suite2p with detection on functional channel

In [ ]:

db_paths = suite2p.run_s2p(settings=settings, db=db)


visualize green and red channel

In [ ]:
reg_outputs = np.load(Path(db["save_path0"]) / "suite2p/plane0/reg_outputs.npy", allow_pickle=True).item()
import matplotlib.pyplot as plt 
from cellpose.transforms import normalize99
redchan = np.clip(normalize99(reg_outputs["meanImg_chan2"]), 0, 1)
greenchan = np.clip(normalize99(reg_outputs["meanImg"]), 0, 1)
rgb = np.stack((redchan, greenchan, redchan), axis=-1)

plt.figure(figsize=(14,8))
plt.subplot(1,3,1)
plt.imshow(greenchan)
plt.title("green (func)")
plt.axis("off")
plt.subplot(1,3,2)
plt.imshow(redchan)
plt.title("red (anat)")
plt.axis("off")
plt.subplot(1,3,3)
plt.imshow(rgb)
plt.title("overlay")
plt.axis("off")

## run ROI detection in second channel 

only use this if you have **two** functional indicators, e.g. gcamp + rgeco

In [ ]:
from suite2p.pipeline_s2p import pipeline
from suite2p.io import BinaryFile
import torch
import shutil 

db0 = np.load(Path(db["save_path0"]) / "suite2p/db.npy", allow_pickle=True).item()
nfolds = db0["nplanes"] * db0.get("nrois", 1)

for i in range(nfolds):
    db_green = np.load(Path(db0["save_path0"]) / f"suite2p/plane{i}/db.npy", allow_pickle=True).item()
    Ly, Lx, n_frames = db_green["Ly"], db_green["Lx"], db_green["nframes"]

    # swap reg files
    reg_file = db_green["reg_file_chan2"]
    reg_file_chan2 = db_green["reg_file"]

    # new save path
    save_path = Path(db["save_path0"]) / "suite2p_red" 
    save_path.mkdir(exist_ok=True)
    (save_path / f"plane{i}").mkdir(exist_ok=True)
    
    # update db 
    db_red = db_green 
    db_red["reg_file"] = reg_file
    db_red["reg_file_chan2"] = reg_file_chan2
    np.save(save_path / f"plane{i}/db.npy", db_red)
    np.save(save_path / f"plane{i}/settings.npy", settings)

    # swap meanImg/meanImg_chan2 in reg_outputs for gui vis 
    reg_outputs = np.load(Path(db["save_path0"]) / f"suite2p/plane{i}/reg_outputs.npy", allow_pickle=True).item()
    reg_outputs_red = reg_outputs.copy()
    reg_outputs_red["meanImg"] = reg_outputs["meanImg_chan2"]
    reg_outputs_red["meanImg_chan2"] = reg_outputs["meanImg"]
    np.save(save_path / f"plane{i}/reg_outputs.npy", reg_outputs_red)
        
    with BinaryFile(Ly=Ly, Lx=Lx, filename=reg_file, n_frames=n_frames) as f_reg, \
        BinaryFile(Ly=Ly, Lx=Lx, filename=reg_file_chan2, n_frames=n_frames)  as f_reg_chan2:
        out = pipeline(str(save_path / f"plane{i}"), f_reg=f_reg, f_reg_chan2=f_reg_chan2, 
                    run_registration=False, settings=settings,
                    device=torch.device(settings["torch_device"]))

    # outputs from detection on chan2 reg file 
    (reg_outputs, detect_outputs, stat, F, Fneu, F_chan2, Fneu_chan2, 
                spks, iscell, redcell, zcorr, plane_times) = out